<h1>Text Mining - Exctracting Dates from Medical Data</h1>
<b>Éverton Bin</b><br>

<h3>Introduction</h3>
<p>
    Through this notebook, we are going to extract dates from medical records anotated in <i>dates.txt</i>, in which each line corresponds to one medical note.
</p>
<p>
    This data was made available by <a href = "https://umich.edu/">University of Michigan</a> along with the <a href = "https://www.coursera.org/learn/python-text-mining">Applied Text Mining in Python</a> course offered through the <b>Coursera</b> platform.
</p>
<p>
    Our goal is to extract one date from each record, regarding their format, standardizing them to one single format (in this case <b>dd/mm/yyy</b>) and then ordering this dates in <b>ascending</b> order, and keeping the original indexes. Some previous assumptions:
    <li>all dates in xx/xx/xx format are mm/dd/yy;</li>
    <li>all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989);</li>
    <li>if the day is missing (e.g. 9/2009), it is going to be assumed as the first day of the month (e.g. September 1, 2009);</li>
    <li>if the month is missing (e.g. 2010), it is going to be assumed as the first month of that year (e.g. January 1, 2010).</li>
</p>

<h3>Loading Data</h3>

In [1]:
# Loading packages:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Opening file and saving records in a list:
doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

# Transforming the list to a Series:
df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [2]:
len(df)

500

<p>
    There are <b>500</b> medical records, each one with one date of interest. Dates are written in several different formats and our mission is to extract all of them.
</p>
<p>
    Let's check some random records, in order to better understand their content:
</p>

In [3]:
df.iloc[5]

'.Per 7/06/79 Movement D/O note:\n'

In [4]:
df.iloc[118]

'see 4/27/2006 consult note Dr. GuevaraWhat factors in prior treatment were helpful/not helpful:\n'

In [5]:
df.iloc[196]

".Feb 18, 1994: made a phone call to Mom and Mom commented that he was talking very fast, hard to interrupt, but was in super happy spirits, so didn't make a big deal of it.\n"

In [6]:
df.iloc[250]

') He endorsed the following hyperarousal symptoms: disturbed sleep (e.g., months with great sleep and then has nights he barely sleeps- mind racing with worries related to job, expecting a child (May 2005)); reckless or self-destructive behavior (e.g., see above); difficulty with concentration; some hypervigilance; and exaggerated startle response triggered by loud noises.\n'

In [7]:
df.iloc[300]

'.Since January 1994, she feels that hse has been much more irritable and frustrated towards her husband about 1 week prior to her menses. She does not have a h/o premenstrual mood symptoms.  She notes that little things would set her off with him. She was not this way at work or in other social settings.\n'

In [8]:
df.iloc[356]

'. Patient states that she will follow up with the Cotta Hospital clinic regarding getting a therapist for ongoing care. (Per LMR, pt transitioned to a new therapist in 9/2003 and was seen for 1 appointment; the pt canceled her f/u appt 2 weeks later and was not seen for f/u appointments since. )This visit for a one-time consultation only? Yes\n'

In [9]:
df.iloc[496]

'therapist and friend died in ~2006 Parental/Caregiver obligations:\n'

<h3>Defining Rules and Extracting Dates</h3>

In [10]:
# Extracting format 0 (04/20/2009; 04/20/09; 4/20/09; 4/3/09):
df0 = df.str.extract(r'(?P<month0>\d{1,2})[/-](?P<day0>\d{1,2})[/-](?P<year0>\d{2,4})')

# Extracting format 1 (Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009):
df1 = df.str.extract(r'(?P<month1>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-,. ]+(?P<day1>\d{1,2})[-,. ]+(?P<year1>\d{2,4})')

# Extracting format 2 (20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009):
df2 = df.str.extract(r'(?P<day2>\d{1,2}) (?P<month2>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[,]* (?P<year2>\d{2,4})')

# Extracting format 3 (Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009):
df3 = df.str.extract(r'(?P<month3>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* (?P<day3>\d{1,2})[st]*[nd]*[th]*[,]* (?P<year3>\d{2,4})')

# Extracting format 4 (Feb 2009; September 2009; Oct, 2010):
df4 = df.str.extract(r'(?P<month4>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[,]* (?P<year4>\d{2,4})')

# Extracting format 5 (6/2008; 12/2009):
df5 = df.str.extract(r'(?P<month5>\d{1,2})/(?P<year5>\d{4})')

# Extracting format 6 (2009; 2010):
df6 = df.str.extract(r'(?P<year6>19\d{2}|20\d{2})')

<h3>Grouping Days, Months and Years</h3>

In [11]:
# Selecting day, month and year
days = pd.DataFrame()
months = pd.DataFrame()
years = pd.DataFrame()

In [12]:
# Days:
days['day0'] = df0['day0']
days['day1'] = df1['day1']
days['day2'] = df2['day2']
days['day3'] = df3['day3']
days['Day'] = 0
days.head()

,day0,day1,day2,day3,Day
0,25,NaN,NaN,NaN,0
1,18,NaN,NaN,NaN,0
2,8,NaN,NaN,NaN,0
3,27,NaN,NaN,NaN,0
4,6,NaN,NaN,NaN,0


In [13]:
# Months:
months['month0'] = df0['month0']
months['month1'] = df1['month1']
months['month2'] = df2['month2']
months['month3'] = df3['month3']
months['month4'] = df4['month4']
months['month5'] = df5['month5']
months['Month'] = 0
months.head()

,month0,month1,month2,month3,month4,month5,Month
0,03,NaN,NaN,NaN,NaN,NaN,0
1,6,NaN,NaN,NaN,NaN,NaN,0
2,7,NaN,NaN,NaN,NaN,NaN,0
3,9,NaN,NaN,NaN,NaN,NaN,0
4,2,NaN,NaN,NaN,NaN,NaN,0


In [14]:
# Years:
years['year0'] = df0['year0']
years['year1'] = df1['year1']
years['year2'] = df2['year2']
years['year3'] = df3['year3']
years['year4'] = df4['year4']
years['year5'] = df5['year5']
years['year6'] = df6
years['Year'] = 0
years.head()

,year0,year1,year2,year3,year4,year5,year6,Year
0,93,NaN,NaN,NaN,NaN,NaN,NaN,0
1,85,NaN,NaN,NaN,NaN,NaN,NaN,0
2,71,NaN,NaN,NaN,NaN,NaN,NaN,0
3,75,NaN,NaN,NaN,NaN,NaN,NaN,0
4,96,NaN,NaN,NaN,NaN,NaN,NaN,0


<h3>Transformations</h3>

In [15]:
# Creating a function to select one valid number for each day, month and year:
def define_date(row):
    '''
    It compares values from a row and chooses a valid one.
    INPUT:
    row: row from a dataframe
    
    OUTPUT:
    row: transformed row from the dataframe
    '''
    values = list()
    for i in range(0, (len(row))):
        if row[i] != 0:
            values.append(row[i])
    if len(values) == 0:
        row[len(row)-1] = 1
    elif len(values) == 1:
        row[len(row)-1] = values[0]
    else:
        row[len(row)-1] = values[0]
    return row

In [16]:
# Filling Day column with the final day:
days = days.fillna(value = 0)
days = days.apply(define_date, axis = 1)

In [17]:
# Filling Month column with the final month:
months = months.fillna(value = 0)
months = months.apply(define_date, axis = 1)

In [18]:
# Filling Year column with the final year:
years = years.fillna(value = 0)
years = years.apply(define_date, axis = 1)

In [19]:
# Transforming all days in two digits format:
for i in range(0, len(days)):
    if len(str(days['Day'].iloc[i])) == 1:
        days['Day'].iloc[i] = '0'+str(days['Day'].iloc[i])

In [20]:
# Transforming months to numeric:
months_dic = {'Jan': '1', 'Feb': '2', 'Mar':'3', 'Apr': '4', 'May': '5', 'Jun': '6', 'Jul': '7', 
              'Aug': '8', 'Sep': '9', 'Oct': '10', 'Nov': '11', 'Dec': '12'}
for key, value in months_dic.items():
    for i in range(0, len(months)):
        if months['Month'].iloc[i] == key:
            months['Month'].iloc[i] = value
            
# Transforming all months in two digits format:
for i in range(0, len(months)):
    if len(str(months['Month'].iloc[i])) == 1:
        months['Month'].iloc[i] = '0'+str(months['Month'].iloc[i])

In [21]:
# Adding digits to the years with length 2:
for i in range(0, len(years)):
    if len(years['Year'].iloc[i]) == 2:
        years['Year'].iloc[i] = '19'+years['Year'].iloc[i]

In [22]:
# Creating a dataframe with the separated final date:
final_date = pd.DataFrame()
final_date['Day'] = days['Day']
final_date['Month'] = months['Month']
final_date['Year'] = years['Year']

In [23]:
# Creating a column for the full date:
final_date['Date'] = None

# Defining a function to create the full date in format dd/mm/yyy:
def joining_date(row):
    '''
    It concatenates columns into one final column date
    INPUT:
    row: recieves one dataframe row
    OUTPUT:
    row: returns the same row with the final column date updated
    '''
    row[3] = str(row[0]) + '/' + str(row[1]) + '/' + str(row[2])
    return row

# Applying function to dataframe:
final_date = final_date.apply(joining_date, axis = 1)
final_date.head()

,Day,Month,Year,Date
0,25,03,1993,25/03/1993
1,18,06,1985,18/06/1985
2,08,07,1971,08/07/1971
3,27,09,1975,27/09/1975
4,06,02,1996,06/02/1996


In [24]:
# Transforming to datetime type:
final_date['Date'] = pd.to_datetime(final_date['Date'], format = '%d/%m/%Y')

# Sorting values by date (ascending):
final_date['Index'] = final_date.index
final_date = final_date.sort_values(['Date', 'Index'], ascending = True)

final_date.head()

,Day,Month,Year,Date,Index
9,10,04,1971,1971-04-10,9
84,18,05,1971,1971-05-18,84
2,08,07,1971,1971-07-08,2
53,11,07,1971,1971-07-11,53
28,12,09,1971,1971-09-12,28


In [25]:
# Transforming final_date index into a pandas series:
ordered_dates_index = pd.Series(final_date.index)
ordered_dates_index

0        9
1       84
2        2
3       53
4       28
      ... 
495    427
496    141
497    186
498    161
499    413
Length: 500, dtype: int64